<a href="https://colab.research.google.com/github/shudhanshumishra/DeepLearning/blob/main/Keras%20tuner/Keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd

In [19]:
df = pd.read_csv("/content/diabetes.csv")

In [20]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [21]:
df.corr()['Outcome']


,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [22]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [23]:
from sklearn.preprocessing  import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2 , random_state=1)

In [25]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [26]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.fit(x_train,y_train ,epochs=100,batch_size=32 , validation_data=(x_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.4043 - loss: 0.7253 - val_accuracy: 0.5649 - val_loss: 0.6766
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6224 - loss: 0.6661 - val_accuracy: 0.7208 - val_loss: 0.6289
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6948 - loss: 0.6253 - val_accuracy: 0.7468 - val_loss: 0.5938
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7484 - loss: 0.5784 - val_accuracy: 0.7727 - val_loss: 0.5656
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7506 - loss: 0.5570 - val_accuracy: 0.7727 - val_loss: 0.5441
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7414 - loss: 0.5407 - val_accuracy: 0.7662 - val_loss: 0.5258
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7525 - loss: 0.5295 - val_accuracy: 0.7597 - val_loss: 0.5129
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7495 - loss: 0.5234 - val_accuracy: 0.7662 - 

# **Keras Tuner**

In [28]:
!pip install -U keras-tuner

In [29]:
import keras_tuner as kt


In [30]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [31]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [32]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 17s


In [33]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [34]:
tuner.get_best_models(num_models=1)[0].summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6 , validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8090 - loss: 0.3988 - val_accuracy: 0.8052 - val_loss: 0.4609
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8185 - loss: 0.4164 - val_accuracy: 0.8117 - val_loss: 0.4601
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7667 - loss: 0.4377 - val_accuracy: 0.8117 - val_loss: 0.4595
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8026 - loss: 0.4260 - val_accuracy: 0.8117 - val_loss: 0.4595
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8088 - loss: 0.4001 - val_accuracy: 0.8117 - val_loss: 0.4602
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.7901 - loss: 0.4257 - val_accuracy: 0.8117 - val_loss: 0.4596
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7775 - loss: 0.4348 - val_accuracy: 0.8117 - val_loss: 0.4596
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8063 - loss: 0.4068 - val_accuracy:

In [36]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [37]:
# prompt: run this as tuner

tuner2 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir2',project_name='myproject3')

tuner2.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))


Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 14s


In [38]:
# prompt: best hyperparameter

tuner2.get_best_hyperparameters()[0].values

{'units': 86, 'optimizer': 'adam'}

In [39]:
# prompt: best mode

tuner2.get_best_models(num_models=1)[0].summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 86)             │           774 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            87 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 861 (3.36 KB)

 Trainable params: 861 (3.36 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model= tuner2.get_best_models(num_models=1)[0]

In [41]:
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6 , validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7466 - loss: 0.5104 - val_accuracy: 0.7857 - val_loss: 0.4875
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7799 - loss: 0.5046 - val_accuracy: 0.7792 - val_loss: 0.4775
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7853 - loss: 0.4722 - val_accuracy: 0.7792 - val_loss: 0.4700
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7570 - loss: 0.4843 - val_accuracy: 0.7792 - val_loss: 0.4674
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7657 - loss: 0.4764 - val_accuracy: 0.7922 - val_loss: 0.4655
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7432 - loss: 0.4838 - val_accuracy: 0.7922 - val_loss: 0.4646
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7749 - loss: 0.4544 - val_accuracy: 0.7922 - val_loss: 0.4630
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7739 - loss: 0.4613 - val_accuracy: 0.79

In [42]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(97,activation='relu',input_dim=8))
  for i in range (hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(97,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer= 'rmsprop' ,loss='binary_crossentropy',metrics=['accuracy'])
  return model

tuner3 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir3',project_name='myproject4')

tuner3.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))


Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 21s


In [43]:
tuner3.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [44]:
tuner3.get_best_models(num_models=1)[0].summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 97)             │           873 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 97)             │         9,506 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 97)             │         9,506 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 97)             │         9,506 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 97)             │         9,506 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            98 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,995 (152.32 KB)

 Trainable params: 38,995 (152.32 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model= tuner3.get_best_models(num_models=1)[0]

In [46]:
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6 , validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7612 - loss: 0.4784 - val_accuracy: 0.8247 - val_loss: 0.4810
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7993 - loss: 0.4347 - val_accuracy: 0.7987 - val_loss: 0.4827
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8205 - loss: 0.3915 - val_accuracy: 0.8052 - val_loss: 0.4782
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8193 - loss: 0.3907 - val_accuracy: 0.7597 - val_loss: 0.5378
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8022 - loss: 0.4144 - val_accuracy: 0.7792 - val_loss: 0.5161
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8160 - loss: 0.3939 - val_accuracy: 0.7922 - val_loss: 0.4864
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8141 - loss: 0.3995 - val_accuracy: 0.8052 - val_loss: 0.5066
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8407 - loss: 0.3595 - val_accuracy:

In [51]:
import keras_tuner as kt
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

def build_model(hp):
    model = Sequential()
    # Define the first layer outside the loop since it has input_dim
    model.add(
        Dense(
            units=hp.Int('units_0', min_value=8, max_value=128, step=8),
            activation=hp.Choice('activation_0', values=['relu', 'tanh', 'sigmoid']),
            input_dim=8
        )
    )

    # Define subsequent hidden layers in the loop
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(
            Dense(
                units=hp.Int(f'units_{i+1}', min_value=8, max_value=128, step=8),
                activation=hp.Choice(f'activation_{i+1}', values=['relu', 'tanh', 'sigmoid'])
            )
        )

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [52]:
tuner4 = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='mydir4',project_name='myproject5')
tuner4.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

tuner4.get_best_hyperparameters()[0].values
tuner4.get_best_models(num_models=1)[0].summary()
model= tuner4.get_best_models(num_models=1)[0]
model.fit(x_train,y_train,epochs=100,batch_size=32,initial_epoch=6 , validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 29s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         1,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │         3,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 56)             │         3,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         1,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 104)            │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 24)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,473 (83.88 KB)

 Trainable params: 21,473 (83.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7523 - loss: 0.4815 - val_accuracy: 0.7662 - val_loss: 0.4865
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7482 - loss: 0.5186 - val_accuracy: 0.7662 - val_loss: 0.5031
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7521 - loss: 0.4923 - val_accuracy: 0.8247 - val_loss: 0.4794
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7269 - loss: 0.5252 - val_accuracy: 0.7792 - val_loss: 0.4734
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7698 - loss: 0.4744 - val_accuracy: 0.7987 - val_loss: 0.4702
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7936 - loss: 0.4536 - val_accuracy: 0.7273 - val_loss: 0.5306
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7146 - loss: 0.5223 - val_accuracy: 0.7922 - val_loss: 0.4771
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7935 - loss: 0.4625 - val_accuracy: 0.81

In [56]:
tuner4 = kt.RandomSearch(build_model,
                         objective='val_accuracy',
                         max_trials=10, # Feel free to adjust the number of trials
                         directory='mydir4',
                         project_name='myproject5')

tuner4.search(x_train, y_train, epochs=20, validation_data=(x_test, y_test)) # Adjust epochs for search as needed

Reloading Tuner from mydir4/myproject5/tuner0.json


In [57]:
# Get the best hyperparameters
best_hps = tuner4.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The optimal number of layers is {best_hps.get('num_layers')}.
The optimal units in the first layer are {best_hps.get('units_0')}.
The optimal activation in the first layer is {best_hps.get('activation_0')}.
""")
# You can also print the optimal units and activation for each additional layer:
for i in range(best_hps.get('num_layers')):
    print(f"The optimal units in layer {i+1} are {best_hps.get(f'units_{i+1}')}.")
    print(f"The optimal activation in layer {i+1} are {best_hps.get(f'activation_{i+1}')}.")

print(f"\nThe optimal optimizer is {best_hps.get('optimizer')}.")


# Get the best model
best_model = tuner4.get_best_models(num_models=1)[0]

# Summarize the best model
best_model.summary()


The optimal number of layers is 8.
The optimal units in the first layer are 32.
The optimal activation in the first layer is sigmoid.

The optimal units in layer 1 are 40.
The optimal activation in layer 1 are tanh.
The optimal units in layer 2 are 80.
The optimal activation in layer 2 are tanh.
The optimal units in layer 3 are 64.
The optimal activation in layer 3 are sigmoid.
The optimal units in layer 4 are 56.
The optimal activation in layer 4 are sigmoid.
The optimal units in layer 5 are 32.
The optimal activation in layer 5 are tanh.
The optimal units in layer 6 are 24.
The optimal activation in layer 6 are relu.
The optimal units in layer 7 are 104.
The optimal activation in layer 7 are tanh.
The optimal units in layer 8 are 24.
The optimal activation in layer 8 are tanh.

The optimal optimizer is adam.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 40)             │         1,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │         3,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 56)             │         3,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         1,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 104)            │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 24)             │         2,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,473 (83.88 KB)

 Trainable params: 21,473 (83.88 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# Train the best model
history = best_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.7527 - loss: 0.5059 - val_accuracy: 0.7662 - val_loss: 0.5145
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7653 - loss: 0.4772 - val_accuracy: 0.7922 - val_loss: 0.4753
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7819 - loss: 0.4968 - val_accuracy: 0.7922 - val_loss: 0.4910
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7613 - loss: 0.4845 - val_accuracy: 0.7468 - val_loss: 0.5185
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7515 - loss: 0.5162 - val_accuracy: 0.7922 - val_loss: 0.4690
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7611 - loss: 0.4859 - val_accuracy: 0.7792 - val_loss: 0.4715
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7801 - loss: 0.4714 - val_accuracy: 0.8052 - val_loss: 0.4802
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7438 - loss: 0.4998 - val_accuracy: 0.7987 -

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# Load dataset
df = pd.read_csv('/content/diabetes.csv')

# Split features and target
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model builder function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    # Tune number of hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=16),
                               activation=hp.Choice('activation', ['relu', 'tanh'])))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])

    return model

# Keras Tuner: Hyperband search
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='keras_tuner_dir',
    project_name='diabetes_classification'
)

# Early stopping
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Search for best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
Best hyperparameters:
- Layers: {best_hps.get('num_layers')}
- Units/layer: {[best_hps.get(f'units_{i}') for i in range(best_hps.get('num_layers'))]}
- Activation: {best_hps.get('activation')}
- Learning rate: {best_hps.get('learning_rate')}
""")

# Train best model
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


Trial 30 Complete [00h 00m 07s]
val_accuracy: 0.772357702255249

Best val_accuracy So Far: 0.7967479825019836
Total elapsed time: 00h 02m 04s

Best hyperparameters:
- Layers: 1
- Units/layer: [80]
- Activation: tanh
- Learning rate: 0.001

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4026 - loss: 0.7320 - val_accuracy: 0.6829 - val_loss: 0.6332
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7139 - loss: 0.6010 - val_accuracy: 0.7236 - val_loss: 0.5589
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7328 - loss: 0.5370 - val_accuracy: 0.7154 - val_loss: 0.5218
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7525 - loss: 0.5181 - val_accuracy: 0.7561 - val_loss: 0.5028
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7434 - loss: 0.5037 - val_accuracy: 0.7642 - val_loss: 0.4891
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7750 - loss: 0.4658 - val_accuracy: 0.7805 - val_loss: 0.4800
Epo